In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None

import matplotlib.pyplot as plt
%matplotlib inline

import datetime as dt
import pickle
import os
import time

import seaborn as sns
sns.set()
from matplotlib import style
# style.use('ggplot')
%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv('sp500_wti.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197718 entries, 0 to 1197717
Data columns (total 34 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   Date                       1197718 non-null  datetime64[ns]
 1   Adj Close                  1197718 non-null  float64       
 2   volume_obv                 1197718 non-null  float64       
 3   volume_fi                  1197718 non-null  float64       
 4   volatility_bbm             1197718 non-null  float64       
 5   volatility_bbw             1197718 non-null  float64       
 6   volatility_bbhi            1197718 non-null  float64       
 7   volatility_bbli            1197718 non-null  float64       
 8   trend_macd                 1197718 non-null  float64       
 9   trend_macd_signal          1197718 non-null  float64       
 10  trend_macd_diff            1197718 non-null  float64       
 11  trend_psar_up_indicator    1197718 no

In [3]:
df.head()

,Date,Adj Close,volume_obv,volume_fi,volatility_bbm,volatility_bbw,volatility_bbhi,volatility_bbli,trend_macd,trend_macd_signal,trend_macd_diff,trend_psar_up_indicator,trend_psar_down_indicator,momentum_ao,momentum_roc,Return,target_return,ticker,exchange,sector,industry,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,2010-01-05,0.989138,-0.097104,-0.009849,0.994569,0.021725,0.0,0.0,-0.014311,-0.003872,0.009048,0.0,0.0,0.000000,-2.492011,-1.086260,-0.355297,A,NYSE,CAPITAL GOODS,BIOTECHNOLOGY: LABORATORY ANALYTICAL INSTRUMENTS,2010,1,1,5,1,5,False,False,False,False,False,False,1262649600
1,2010-01-06,0.985623,-0.947242,-0.009849,0.991587,0.024478,0.0,0.0,-0.014311,-0.003872,0.009048,0.0,1.0,0.000000,-2.492011,-0.355297,-0.129668,A,NYSE,CAPITAL GOODS,BIOTECHNOLOGY: LABORATORY ANALYTICAL INSTRUMENTS,2010,1,1,6,2,6,False,False,False,False,False,False,1262736000
2,2010-01-07,0.984345,-1.758433,-0.009849,0.989776,0.024632,0.0,0.0,-0.014311,-0.003872,0.009048,0.0,0.0,0.000000,-2.492011,-0.129668,-0.032455,A,NYSE,CAPITAL GOODS,BIOTECHNOLOGY: LABORATORY ANALYTICAL INSTRUMENTS,2010,1,1,7,3,7,False,False,False,False,False,False,1262822400
3,2010-01-08,0.984025,-2.737046,-0.009849,0.988626,0.023876,0.0,0.0,-0.014311,-0.003872,0.009048,0.0,0.0,0.000000,-2.492011,-0.032455,0.064939,A,NYSE,CAPITAL GOODS,BIOTECHNOLOGY: LABORATORY ANALYTICAL INSTRUMENTS,2010,1,1,8,4,8,False,False,False,False,False,False,1262908800
4,2010-01-11,0.984664,-1.483868,-0.009849,0.987966,0.022581,0.0,0.0,-0.014311,-0.003872,0.009048,0.0,0.0,-0.003062,-2.492011,0.064939,-1.200519,A,NYSE,CAPITAL GOODS,BIOTECHNOLOGY: LABORATORY ANALYTICAL INSTRUMENTS,2010,1,2,11,0,11,False,False,False,False,False,False,1263168000


In [23]:
df_temp = df.loc[df.ticker=='AAPL', ['Date', 'Return']]
df_temp = df_temp.rename(columns={'Return':'AAPL'})
df_temp = df_temp.set_index('Date')
df_temp

,AAPL
Date,
2010-01-05,0.172887
2010-01-06,-1.590633
2010-01-07,-0.184858
2010-01-08,0.664833
2010-01-11,-0.882160
2010-01-12,-1.137500
2010-01-13,1.410553
2010-01-14,-0.579160
2010-01-15,-1.671203


In [11]:
dates = df.Date.unique()
is_ascending = lambda A : np.all(A[:-1] < A[1:])
is_ascending(dates)

True

In [56]:
df_rets = pd.DataFrame(index=dates)
tickers = df.ticker.unique().tolist()
t0 = time.time()
for i, ticker in enumerate(tickers):
    df_temp = df.loc[df.ticker==ticker, ['Date', 'Return']]
    df_temp = df_temp.rename(columns={'Return':ticker})
    df_temp = df_temp.set_index('Date')
    df_rets = df_rets.join(df_temp)
    if i % 50 == 0 or i == len(tickers)-1:
        print(i, " Elapsed: ", time.time() - t0)
df_rets = df_rets.reset_index()
df_rets = df_rets.rename(columns={'index': 'Date'})
df_rets.to_csv('data/daily_rets.csv', index=False)

0  Elapsed:  0.03928041458129883
50  Elapsed:  2.0759084224700928
100  Elapsed:  4.138542652130127
150  Elapsed:  6.253758668899536
200  Elapsed:  8.400441646575928
250  Elapsed:  10.596253156661987
300  Elapsed:  12.834295988082886
350  Elapsed:  15.108702182769775
400  Elapsed:  17.441991806030273
450  Elapsed:  19.817248821258545
484  Elapsed:  21.64908003807068


In [97]:
np.isnan(df_rets2.iloc[0]['ABBV'])

True

In [95]:
# df_rets2 = pd.read_csv('data/daily_rets.csv', index_col='Date', parse_dates=True, na_values=['NaN'])
df_rets2 = pd.read_csv('data/daily_rets.csv', na_values=['NaN'])
df_rets2['Date'] = pd.to_datetime(df_rets2['Date'])
df_rets2.head()

,Date,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADS,ADSK,AEE,AEP,AES,AFL,AGN,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMD,AME,AMG,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ARNC,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BIIB,BK,BKNG,BLK,BLL,BMY,BR,BSX,BWA,BXP,C,CAG,CAH,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CELG,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA,CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTXS,CVS,CVX,CXO,D,DAL,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DRE,DRI,DTE,DUK,DVA,DVN,DXC,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOG,HOLX,HON,HP,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,IBM,ICE,IDXX,IEX,IFF,ILMN,INCY,INFO,INTC,INTU,IP,IPG,IPGP,IQV,IR,IRM,ISRG,IT,ITW,IVZ,JBHT,JCI,JEC,JKHY,JNJ,JNPR,JPM,JWN,K,KEY,KEYS,KHC,KIM,KLAC,KMB,KMI,KMX,KO,KR,KSS,KSU,L,LB,LDOS,LEG,LEN,LH,LKQ,LLY,LMT,LNC,LNT,LOW,LRCX,LUV,LW,LYB,M,MA,MAA,MAC,MAR,MAS,MCD,MCHP,MCK,MCO,MDLZ,MDT,MET,MGM,MHK,MKC,MKTX,MLM,MMC,MMM,MNST,MO,MOS,MPC,MRK,MRO,MS,MSCI,MSFT,MSI,MTB,MTD,MU,MXIM,MYL,NBL,NCLH,NDAQ,NEE,NEM,NFLX,NI,NKE,NKTR,NLSN,NOC,NOV,NRG,NSC,NTAP,NTRS,NUE,NVDA,NVR,NWL,NWS,NWSA,O,OKE,OMC,ORCL,ORLY,OXY,PAYX,PBCT,PCAR,PEG,PEP,PFE,PFG,PG,PGR,PH,PHM,PKG,PKI,PLD,PM,PNC,PNR,PNW,PPG,PPL,PRGO,PRU,PSA,PSX,PVH,PWR,PXD,PYPL,QCOM,QRVO,RCL,RE,REG,REGN,RF,RHI,RJF,RL,RMD,ROK,ROL,ROP,ROST,RSG,RTN,SBAC,SBUX,SCHW,SEE,SHW,SIVB,SJM,SLB,SLG,SNA,SNPS,SO,SPG,SPGI,SRE,STT,STX,STZ,SWK,SWKS,SYF,SYK,SYY,T,TAP,TDG,TEL,TFX,TGT,TIF,TJX,TMO,TMUS,TPR,TRIP,TROW,TRV,TSCO,TSN,TTWO,TWTR,TXN,TXT,UA,UAA,UAL,UDR,UHS,ULTA,UNH,UNM,UNP,UPS,URI,USB,UTX,V,VAR,VFC,VLO,VMC,VNO,VRSK,VRSN,VRTX,VTR,VZ,WAB,WAT,WBA,WDC,WEC,WELL,WFC,WHR,WLTW,WM,WMB,WMT,WRK,WU,WY,WYNN,XEC,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
0,2010-01-05,-1.086260,11.320754,-0.594358,0.172887,NaN,-0.713476,-2.402746,-0.807929,0.618023,1.644650,-0.157876,0.540197,-0.537005,0.197294,-1.519281,-0.396256,-1.144813,-1.024143,2.900990,-0.992806,-1.873535,2.506269,2.984084,-0.537397,2.970681,-0.183774,-2.648647,5.477488,1.644196,NaN,-1.430049,-0.769235,0.103095,-0.440416,0.798495,-0.866251,2.331997,1.656699,0.589999,NaN,-2.240143,1.087869,-0.632572,-1.277738,1.180693,-0.829726,-2.089680,NaN,-1.054285,-3.123121,-1.050140,0.176987,-0.751232,0.742309,0.651294,1.015451,-0.219932,-0.835279,3.275187,3.250486,-0.686226,2.537946,-0.928748,0.777774,-0.484710,1.060443,1.160917,0.431721,-1.124024,-1.560670,-0.396651,0.332960,4.766136,1.520126,3.823527,0.432707,-0.891476,1.195561,-2.252893,NaN,2.387843,0.959356,0.966929,-0.980391,NaN,2.224900,-1.166131,5.909581,1.622389,NaN,-1.353117,-2.072800,NaN,1.677208,0.037737,0.808688,1.090669,0.360658,-1.355330,-0.081773,1.343352,2.364722,-0.316657,-3.179726,-0.757580,3.943664,-0.129789,1.165559,0.095095,-0.302980,NaN,0.177152,-0.027460,-0.427693,-0.445527,2.640198,-0.151569,0.163670,1.132484,0.070024,-0.394182,0.708329,-3.502848,-1.232032,7.843138,-0.374604,3.353872,0.779897,-2.240024,3.584235,-0.119760,-0.249454,3.935486,4.404622,-0.613792,0.658022,-1.266352,-0.472145,1.486377,-0.428577,-1.125406,-1.649971,-0.851132,0.104482,-0.034642,2.809919,-1.046025,-1.524859,-1.300133,-0.031777,-0.891577,-0.391435,-1.279354,0.161476,-1.225339,-0.930994,-0.389225,-0.349244,0.605180,-2.173911,-0.590798,-1.594650,0.720244,-1.636660,0.684144,0.426029,-0.672834,6.614789,NaN,2.069957,NaN,NaN,0.599089,1.306176,-1.111825,-2.704704,4.282118,0.424841,2.667989,0.245104,1.113289,NaN,-0.334389,NaN,NaN,-0.119352,1.722101,2.166663,NaN,0.158983,0.517801,-0.092381,-0.930894,0.357872,NaN,-0.440367,-0.440366,-0.104907,-0.208841,1.217137,0.248595,1.767967,0.041136,1.279999,-1.106333,2.173911,0.323886,NaN,0.732470,0.617478,6.466929,7.669740,NaN,NaN,0.746271,0.000000,0.198269,5.836114,NaN,0.419446,-1.412

In [70]:
mindate, maxdate = '2013-01-05', '2013-03-05'
date_mask = (df_rets2['Date'] > mindate) & (df_rets2['Date'] < maxdate)
df_rets2.loc[date_mask, ['A']].values.shape

(39, 1)

In [90]:
i = df_rets2.index[df_rets2['Date']=='2013-01-11']
i0 = i - 15

In [91]:
df_rets2.iloc[i0[0]:i[0]+1][['A', 'Date']] # len = 16

,A,Date
745,-1.932836,2012-12-19
746,1.897019,2012-12-20
747,-0.749514,2012-12-21
748,0.048718,2012-12-24
749,-0.438277,2012-12-26
750,-0.611391,2012-12-27
751,-1.993110,2012-12-28
752,2.786843,2012-12-31
753,2.296039,2013-01-02
754,0.358166,2013-01-03


In [105]:
def valid_conv(df_rets, ticker, end_date, length=256):
    i_end = df_rets.index[df_rets['Date'] == end_date]
    i_start = i_end - length + 1
    if (i_start not in df_rets.index) or np.isnan(df_rets.iloc[i_start][ticker]):
        return False
    return True

def get_series(df_rets, ticker, end_date, length=256):
    i_end = df_rets.index[df_rets['Date'] == end_date]
    i_start = i_end - length + 1
    if (i_start not in df_rets.index) or np.isnan(df_rets.iloc[i_start][ticker]):
        return None
    return df_rets.iloc[i_start:i_end][ticker].values

In [103]:
df_rets2.iloc[i0[0]:i[0]+1][['A']].values.shape, df_rets2.iloc[i0[0]:i[0]+1]['A'].values.shape

((16, 1), (16,))